In [1]:
import pandas as pd
import numpy as np

In [2]:
PATH_DATES = '../data/raw/hep-th-slacdates_unzipped.txt'
PATH_CITATION ='../data/raw/hep-th-citations_unzipped.txt'
PATH_LABEL = '../data/processed/papers_label.csv'

## Load data

In [3]:
def read_dates(path):
    df = pd.read_csv(path, sep=' ', header=None, names=['id_paper', 'slac_date'])
    df['slac_date'] = pd.to_datetime(df['slac_date'])
    display(df.head(5))
    return (df)
    

In [4]:
def read_citation(path, df2):
    df1 = pd.read_csv(path, sep=' ', header=None, names=['id_from', 'id_to'])
    # Add dates
    df1 = pd.merge(df1, df2, how="left", left_on='id_from', right_on='id_paper')
    df1 = pd.merge(df1, df2, how="left", left_on='id_to', right_on='id_paper')
    df1.rename(columns={"slac_date_x": "date_from", "slac_date_y": "date_to"}, inplace= True)
    # Select columns
    df1 = df1[['id_from', 'date_from', 'id_to', 'date_to']]
    display(df1.head(5))
    return(df1)
    
    

In [5]:
df_dates = read_dates(PATH_DATES)

,id_paper,slac_date
0,1001,2000-01-04
1,1002,2000-01-04
2,1003,2000-01-04
3,1004,2000-01-04
4,1005,2000-01-04


In [6]:
df_citation = read_citation(PATH_CITATION, df_dates)

,id_from,date_from,id_to,date_to
0,1001,2000-01-04,9304045,1993-04-13
1,1001,2000-01-04,9308122,1993-08-30
2,1001,2000-01-04,9309097,1993-09-24
3,1001,2000-01-04,9311042,1993-11-12
4,1001,2000-01-04,9401139,1994-02-04


In [7]:
df_citation[df_citation['date_from']>=df_citation['date_to']]

,id_from,date_from,id_to,date_to
0,1001,2000-01-04,9304045,1993-04-13
1,1001,2000-01-04,9308122,1993-08-30
2,1001,2000-01-04,9309097,1993-09-24
3,1001,2000-01-04,9311042,1993-11-12
4,1001,2000-01-04,9401139,1994-02-04
...,...,...,...,...
352802,9912293,2000-01-03,9907211,1999-07-30
352803,9912293,2000-01-03,9908105,1999-08-17
352804,9912293,2000-01-03,9908142,1999-08-23
352805,9912293,2000-01-03,9909081,1999-09-14


## Create label

In [10]:
def create_label(df1, df2, path, months=3):
    
    # Define variables
    values = [1, 2]
    ls_future_citations = []
    ls_label = []

    # Form main structure with 27,770 papers
    df3 = pd.DataFrame(pd.concat([df1['id_from'], df1['id_to']]), columns=['id_paper'] )
    df3.drop_duplicates(ignore_index=True, inplace = True )
    df3 = pd.merge(df3, df2, how='inner', on='id_paper')

    #Add label
    for index, row in df3.iterrows():
        date_p  = row['slac_date'] + pd.DateOffset(months=months)
        df4 = df1[df1.date_from <= date_p]
        if df4.empty or row['id_paper'] not in df4.id_to.tolist(): # Not citations during those 3 months or not citations for that particular paper
            ls_future_citations.append(0)
            ls_label.append(0)
        else:
            df4 = df4.groupby(['id_to']).count()[['id_from']].reset_index().rename(columns={'id_from':'n_citations'})
            df4['n_citations_p'] = df4.n_citations.rank(pct = True)
            conditions = [(df4.n_citations_p >= 0.67), (df4.n_citations_p >= 0.33) & (df4.n_citations_p < 0.67)]
            df4['label'] = np.select(conditions, values, default=3)
            ls_future_citations.append(df4[df4.id_to == row['id_paper']].iloc[0,1])
            ls_label.append(df4[df4.id_to == row['id_paper']].iloc[0,3])

    df3['n_future_citations'] = ls_future_citations
    df3['label'] = ls_label
    
    df3.to_csv(path, index=False)
    display(df3.head(5))
    
    return (df3)


In [11]:
df_papers = create_label(df_citation, df_dates, PATH_LABEL)


,id_paper,slac_date,n_future_citations,label
0,1001,2000-01-04,0,0
1,1002,2000-01-04,16,1
2,1003,2000-01-04,10,1
3,1004,2000-01-04,1,3
4,1005,2000-01-04,0,0


In [12]:
df_papers[df_papers['label'] != 0]

,id_paper,slac_date,n_future_citations,label
1,1002,2000-01-04,16,1
2,1003,2000-01-04,10,1
3,1004,2000-01-04,1,3
9,1010,2000-01-05,1,3
10,1011,2000-01-05,2,3
...,...,...,...,...
27724,9903171,1999-03-22,2,3
27732,9905151,1999-05-21,1,3
27740,9905010,1999-05-04,1,3
27742,9907198,1999-07-28,1,3
